# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
***

# Import Modules <a name="import-modules"></a>

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "data"))

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow"))


# Python Modules***************************************************************
# *****************************************************************************
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.plotly as py

# My Modules ******************************************************************
# *****************************************************************************
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot

# Local Imports ***************************************************************
# *****************************************************************************
from an_data_processing import load_df, load_vegge_data

from proj_data_fe_graph import (
    proj_dir_name,
    system_color_map,
    scaling_dict,
    scaling_dict_ideal,
    gas_molec_dict,
    smart_format_dict,   
    smart_format_dict_volcano_scaling,
    system_color_map_vegge,
    )

Divisor is Energy class instance!!!


# Script Inputs

In [3]:
save_plot = True

bias = 0.
scaling_dict = scaling_dict_ideal

smart_format_dict = smart_format_dict_volcano_scaling

# Load Data

In [4]:
%%capture

# My DFT data
df_m = load_df(
    from_file=False,
    root_dir=os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow/adsorption_study"),
    filter_columns=[
        "pdos_data",
        "charge_history",
        "magmom_history",
        ],
    )

# Vegge ACS paper data
df_vegge = load_vegge_data(
    ads_e_mode="w_solvent_correction",
    )

# Instantiate the ORR_Free_E_Plot Instance

In [5]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e", 
    smart_format=None,
    bias=bias,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="ORR",
    )

# Picking the Lowest Energy States

In [6]:
group_params = ["system", "spinpol", "graph_site"]
grouped = df_m.groupby(group_params)

lowest_energy_list = []
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    for j_cnt, df_j in df_i.groupby("adsorbate"):
        series_j = df_j.loc[df_j["ads_e"].idxmin()]
        lowest_energy_list.append(series_j)

df_2 = pd.concat(lowest_energy_list, axis=1, keys=[s.name for s in lowest_energy_list]).T

# Adding ORR data series

In [7]:
prop_name_list = [
    "author_short",
    "system",
    "spinpol",
    "site",
    "graph_site",
     ]

grouped = df_2.groupby(group_params)
for i_ind, (name, df_i) in enumerate(grouped):
#     system_i = "_".join([
#         str(df_i.iloc[0]["system"]),
#         str(df_i.iloc[0]["spinpol"]),
#         str(df_i.iloc[0]["graph_site"]),
#         ])

#     color_i = system_color_map.get(system_i, "pink")

    ORR_PLT.add_series(
        df_i,
        plot_mode="all",
        group="norskov",
        overpotential_type="ORR",
        property_key_list=prop_name_list,
#         color=color_i,
        )

# Processing Vegge Data

In [8]:
prop_name_list = [
    "author_short",
    "system",
    "n_per_ads",
    "graphene_layers",
     ]

group_params = [
    "system",
    "n_per_ads",
    "graphene_layers",
    ]

grouped = df_vegge.groupby(group_params)
for i_ind, (name, df_i) in enumerate(grouped):

#     system_i = "_".join([
#         str(df_i.iloc[0]["system"]),
#         str(df_i.iloc[0]["n_per_ads"]),
#         str(df_i.iloc[0]["graphene_layers"])])

    ORR_PLT.add_series(
        df_i, plot_mode="all", group="vegge",
        overpotential_type="ORR",
        property_key_list=prop_name_list,
#         color=system_color_map_vegge[system_i],
        )

# Plotting

In [9]:
from orr_reaction.orr_fed_plot import Volcano_Plot

VP = Volcano_Plot(
    ORR_PLT,
    x_ax_species="oh",
    smart_format_dict=smart_format_dict,
    )

VP.create_volcano_relations_plot()

volcano_legs_data = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict,

    plot_all_legs=False,
    plot_min_max_legs=True,
    )

In [10]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

## Figure with Legend

In [11]:
layout = VP.get_plotly_layout(
    showlegend=True,
    width=3.3 * 9. * 37.795275591,
    height=5.1 * 9. * 37.795275591,
    )

py.iplot(
    {
        "data": VP.data_points + volcano_legs_data,
        "layout": layout,
        },
    filename=os.path.join(
        save_dir,
        "orr_volcano",
        "orr_volcano_yes_legend_large",
        ),

    )

## Figure without Legend

In [12]:
layout_override_dict = {
    "yaxis": {
        "dtick": 0.4,
        },
    
    "xaxis": {
        "dtick": 0.4,
        },
    }

layout = VP.get_plotly_layout(
    showlegend=False,
    width=1.5 * 7.964 * 37.795275591,
    height=1.5 * 5.6002 * 37.795275591,    
    layout_dict=layout_override_dict,
    )

py.iplot(
    {
        "data": VP.data_points + volcano_legs_data,
        "layout": layout,
        },
    filename=os.path.join(
        save_dir,
        "orr_volcano",
        "orr_volcano_no_legend__main",
        ),
    )

In [13]:
# import os
# import sys

# sys.path.insert(
#     0,
#     os.path.join(
#         os.environ["PROJ_fe_graph"],
#         "data",
#         ),
#     )